# audio-dsp Quickstart Guide

This notebook provides a quick introduction to the `audio-dsp` library, covering:
- Loading and saving audio
- Basic synthesis
- Applying effects
- Visualizing waveforms
- Playing audio in the notebook

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Metallicode/python_audio_dsp/blob/master/examples/Quickstart.ipynb)

## Installation

Run this cell to install the library (required for Google Colab):

In [1]:
!pip install audio-dsp -q

In [ ]:
## Load example .wav files to runtime (for Google Colab)
import os
import requests

# Define where to save the files in Colab
output_dir = "/content/assets"

try:
    os.makedirs(output_dir, exist_ok=True)

    # Define the API URL for the assets folder
    api_url = "https://api.github.com/repos/Metallicode/python_audio_dsp/contents/examples/assets?ref=master"

    # Get the list of files from GitHub
    response = requests.get(api_url)
    
    if response.status_code == 200:
        files = response.json()
        print(f"Found {len(files)} files in 'examples/assets'...")
        
        # Download each file
        for file in files:
            if file['type'] == 'file':
                print(f"   Downloading {file['name']}...")
                raw_url = file['download_url']
                file_content = requests.get(raw_url).content
                
                with open(os.path.join(output_dir, file['name']), 'wb') as f:
                    f.write(file_content)
        
        print("\nSuccess! Files are ready in:", output_dir)
        print("Files:", os.listdir(output_dir))
        
    elif response.status_code == 404:
        print("Error 404: The folder 'examples/assets' was not found.")
    else:
        print(f"GitHub API Error: {response.status_code}")
        
except Exception as e:
    print(f"Download failed: {e}")
    print("Will use synthesized sounds instead.")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Audio, display

# Core imports
from audio_dsp.utils import load_audio, save_audio, normalize_audio
from audio_dsp.synth import SubtractiveSynth
from audio_dsp.effects import fuzz_distortion, filter_effect

SAMPLE_RATE = 44100

# Visualization helper
def plot_waveform(audio, sr, title="Waveform", figsize=(12, 3), color='#2563eb'):
    """Plot a waveform with time axis."""
    plt.figure(figsize=figsize)
    time = np.arange(len(audio)) / sr
    plt.plot(time, audio, color=color, linewidth=0.5)
    plt.xlabel('Time (s)')
    plt.ylabel('Amplitude')
    plt.title(title)
    plt.xlim(0, time[-1])
    plt.ylim(-1.1, 1.1)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

def plot_comparison(audio1, audio2, sr, title1="Original", title2="Processed", figsize=(12, 5)):
    """Plot two waveforms for comparison."""
    fig, axes = plt.subplots(2, 1, figsize=figsize, sharex=True)
    time1 = np.arange(len(audio1)) / sr
    time2 = np.arange(len(audio2)) / sr
    
    axes[0].plot(time1, audio1, color='#2563eb', linewidth=0.5)
    axes[0].set_ylabel('Amplitude')
    axes[0].set_title(title1)
    axes[0].set_ylim(-1.1, 1.1)
    axes[0].grid(True, alpha=0.3)
    
    axes[1].plot(time2, audio2, color='#dc2626', linewidth=0.5)
    axes[1].set_xlabel('Time (s)')
    axes[1].set_ylabel('Amplitude')
    axes[1].set_title(title2)
    axes[1].set_ylim(-1.1, 1.1)
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

## For Google Colab Users

If running on Colab, upload the sample files from the `assets` folder, or skip the loading sections and work with synthesized audio.

## 1. Basic Synthesis

Let's start by creating sounds using the built-in synthesizers.

In [ ]:
# Create a subtractive synth
synth = SubtractiveSynth(sample_rate=SAMPLE_RATE)

# Configure the synth
synth.osc_wave = "saw"          # Oscillator: sine, saw, square, triangle
synth.filter_cutoff = 1200      # Low-pass filter cutoff
synth.filter_resonance = 2.0    # Filter resonance
synth.attack = 0.01             # ADSR envelope
synth.decay = 0.1
synth.sustain = 0.7
synth.release = 0.3

# Synthesize a note (A3 = 220 Hz)
note = synth.synthesize(freq=220, duration=1.0)

print(f"Generated {len(note)} samples ({len(note)/SAMPLE_RATE:.2f}s)")
plot_waveform(note, SAMPLE_RATE, "Synthesized Note (A3 - 220 Hz)")
Audio(note, rate=SAMPLE_RATE)

In [ ]:
# Play a simple melody - C major scale
frequencies = [262, 294, 330, 349, 392, 440, 494, 523]
melody = np.concatenate([synth.synthesize(f, 0.3) for f in frequencies])

plot_waveform(melody, SAMPLE_RATE, "C Major Scale")
Audio(melody, rate=SAMPLE_RATE)

## 2. Applying Effects

Process audio with the effects library.

In [ ]:
# Create a longer synth note to demonstrate effects
synth.osc_wave = "saw"
synth.filter_cutoff = 2000
test_audio = synth.synthesize(freq=110, duration=2.0)

# Apply fuzz distortion
fuzzed = fuzz_distortion(test_audio, gain=8.0, threshold=0.3, mix=0.8)

plot_comparison(test_audio, fuzzed, SAMPLE_RATE, "Original Synth", "With Fuzz Distortion")

print("Original:")
display(Audio(test_audio, rate=SAMPLE_RATE))

print("\nWith Fuzz:")
display(Audio(fuzzed, rate=SAMPLE_RATE))

In [ ]:
# Apply a resonant low-pass filter
filtered = filter_effect(
    test_audio,
    sample_rate=SAMPLE_RATE,
    cutoff=400,
    resonance=4.0,
    filter_type="lowpass"
)

plot_comparison(test_audio, filtered, SAMPLE_RATE, "Original", "Low-pass Filtered (400 Hz, high resonance)")

print("Low-pass filtered:")
Audio(filtered, rate=SAMPLE_RATE)

## 3. Chaining Effects

Combine multiple effects by passing audio through a processing chain.

In [ ]:
from audio_dsp.effects import overdrive_distortion, bitcrush_distortion

# Start with a clean synth
synth.filter_cutoff = 3000
clean = synth.synthesize(freq=165, duration=2.0)  # E2

# Chain: Overdrive -> Bitcrush -> Filter
processed = clean.copy()
processed = overdrive_distortion(processed, gain=3.0, mix=0.6)
processed = bitcrush_distortion(processed, bits=8, mix=0.4)
processed = filter_effect(processed, SAMPLE_RATE, cutoff=2000, resonance=1.5)

# Normalize to prevent clipping
processed = normalize_audio(processed, peak=0.9)

plot_comparison(clean, processed, SAMPLE_RATE, "Clean Synth", "Processed (Overdrive + Bitcrush + Filter)")

print("Clean:")
display(Audio(clean, rate=SAMPLE_RATE))

print("\nProcessed:")
display(Audio(processed, rate=SAMPLE_RATE))

## 4. Visualizing Frequency Content

View the spectrum of your audio.

In [ ]:
def plot_spectrum(audio, sr, title="Frequency Spectrum", figsize=(12, 4)):
    """Plot the frequency spectrum of audio."""
    # Compute FFT
    n = len(audio)
    fft = np.abs(np.fft.rfft(audio))
    freqs = np.fft.rfftfreq(n, 1/sr)
    
    # Convert to dB
    fft_db = 20 * np.log10(fft + 1e-10)
    
    plt.figure(figsize=figsize)
    plt.plot(freqs, fft_db, color='#2563eb', linewidth=0.5)
    plt.xlabel('Frequency (Hz)')
    plt.ylabel('Magnitude (dB)')
    plt.title(title)
    plt.xlim(20, 10000)
    plt.xscale('log')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

# Compare spectrum before and after filtering
print("Spectrum comparison:")
plot_spectrum(test_audio, SAMPLE_RATE, "Original Saw Wave Spectrum")
plot_spectrum(filtered, SAMPLE_RATE, "After Low-pass Filter (400 Hz)")

## 5. Saving Audio

Save your processed audio to a file.

In [ ]:
# Save the processed audio
save_audio("output_quickstart.wav", processed, SAMPLE_RATE)
print("Saved to output_quickstart.wav")

## Next Steps

Explore the other example notebooks:
- **Synthesis.ipynb** - Deep dive into synthesizers
- **Effects.ipynb** - Complete effects reference
- **Sequencing.ipynb** - Algorithmic composition